<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Chord group analysis</strong></div>
</div>

On the hypothesis that the chorus is a subset of the verse we have investigated the difference in the sets represented by the chords found in the verse and in the chorus.

Our analysis have shown that the hypothesis is not confirmed, it is even slighty the opposite. The verse and chorus sharing on average 80% of their chords. The chorus being richer in 40% of the songs, the same can be said for the verses in 30 % of the songs, the rest have a chorus and verse of equal diversity. We have looked into each of these groups to see if we could find any relevant differences.

#### Method
Each song was analysed in a set representation of their chords, the chords were assigned to the group chorus, verse, or both in regards to where they appeared. For instance a chord 7min found in only the chorus would go in the set "chorus_only". The result was three vector with no matching chords. The vector chorus and verse holding the most element gives their name to the majority, in case of equality it is tagged "equal".
For the sake of clarity we chose to simplify the data with the relative chords and three shorthands (maj, min, other).



In [ ]:
#functions to prepare the dataset of subsets

def chord_to_lists(chord_lists):
    array=[]
    for chord_list in chord_lists :
        for chord in chord_list:
            array.append(chord)
    return (list(set(array)))


def length_of_set(df,subset,majority=""):
    if(majority!=""):
        b=pd.DataFrame(df[df["majority"]==majority][subset].map(lambda y: len(y)))
    else:
         b=pd.DataFrame(df[subset].map(lambda y: len(y)))
    b=b[b[subset]!=0]
    return b[subset]


def compare(chorus_verse) :
     '''Compare two lists to create a dictionnary with the lists of the chords found only in the chorus (chorus_only),
     verse (verse_only), both (intersection), the longuest vector is also placed in the section majority, and the ratio
     compares the overlap of the chorus and verse, chorus/verse in case that the verse is more diverse than the chorus and 
     vice versa. '''
        
    chorus=chorus_verse[0].copy()
    verse=chorus_verse[1].copy()
    length_chorus=len(chorus)
    length_verse=len(verse)  
    data={}
    data["intersection"]=[]
    data["chorus_only"]=chorus.copy()
    data["verse_only"]=verse.copy()
    data["majority"]=""
    data["ratio"]=0
    
    if(length_chorus>length_verse) :
        for element in verse :
            if(element in chorus) :
                data["intersection"].append(element)
                data["verse_only"].remove(element)
                data["chorus_only"].remove(element)
        data["ratio"]= len(data["intersection"])/length_verse   
        data["majority"]="chorus"
    else :
        for element in chorus :
            if(element in verse) :
                data["intersection"].append(element)
                data["verse_only"].remove(element)
                data["chorus_only"].remove(element)
        data["ratio"]=len(data["intersection"])/length_chorus
        if(length_chorus==length_verse) :
            data["majority"]="equal"
        else :
            data["majority"]="verse" 
    return data

In [ ]:
#functions to prepare the dataset of subsets

def chord_to_lists(chord_lists):
    array=[]
    for chord_list in chord_lists :
        for chord in chord_list:
            array.append(chord)
    return (list(set(array)))


def length_of_set(df,subset,majority=""):
    if(majority!=""):
        b=pd.DataFrame(df[df["majority"]==majority][subset].map(lambda y: len(y)))
    else:
         b=pd.DataFrame(df[subset].map(lambda y: len(y)))
    b=b[b[subset]!=0]
    return b[subset]


def compare(chorus_verse) :
     '''Compare two lists to create a dictionnary with the lists of the chords found only in the chorus (chorus_only),
     verse (verse_only), both (intersection), the longuest vector is also placed in the section majority, and the ratio
     compares the overlap of the chorus and verse, chorus/verse in case that the verse is more diverse than the chorus and 
     vice versa. '''
        
    chorus=chorus_verse[0].copy()
    verse=chorus_verse[1].copy()
    length_chorus=len(chorus)
    length_verse=len(verse)  
    data={}
    data["intersection"]=[]
    data["chorus_only"]=chorus.copy()
    data["verse_only"]=verse.copy()
    data["majority"]=""
    data["ratio"]=0
    
    if(length_chorus>length_verse) :
        for element in verse :
            if(element in chorus) :
                data["intersection"].append(element)
                data["verse_only"].remove(element)
                data["chorus_only"].remove(element)
        data["ratio"]= len(data["intersection"])/length_verse   
        data["majority"]="chorus"
    else :
        for element in chorus :
            if(element in verse) :
                data["intersection"].append(element)
                data["verse_only"].remove(element)
                data["chorus_only"].remove(element)
        data["ratio"]=len(data["intersection"])/length_chorus
        if(length_chorus==length_verse) :
            data["majority"]="equal"
        else :
            data["majority"]="verse" 
    return data

##### Dataset
A dataset containing the intersection of the sets of unique chords is created for each song.
"Verse_only" denotes the chords only found in the verse, "Chorus_only" in the chorus, and intersection in both. 
The "majority" column indicates which of the previously mentioned category is the most diverse (longest vector). 

In [ ]:
#creates the data frame intersection containing the sub_sets 

chord_lists=beats_collection_df[ (beats_collection_df["section_type"] =="chorus") | (beats_collection_df["section_type"] =="verse") ]
chord_lists=squeeze_section(df)
chord_lists=dfs.groupby(["song_id","section_type"]).agg_chord_label.agg(chord_to_lists)
chord_lists=pd.DataFrame(chord_lists)

intersection=pd.DataFrame(chord_lists.groupby(["song_id"]).agg_chord_label.agg(compare))
for key in ["intersection","majority","ratio","verse_only","chorus_only"]:
    intersection[key]=intersection.agg_chord_label.map(lambda y: y[key])
intersection.drop(columns="agg_chord_label",inplace=True)

In [ ]:
intersection.head(5)

In [ ]:
print(intersection["majority"].value_counts())

40% of the songs find more diversity in the chorus than verse, 30% in the verse, and the rest of the songs are equally distributed between the two.

In [ ]:
data_length=[]
for key in ["intersection","chorus_only","verse_only"] :
    data_length.append(length_of_set(intersection,subset=key))
    
plt.figure(figsize = (14,7))
plt.boxplot(data_length, notch = True, 
                  bootstrap = 5000, 
                  showfliers = True, 
                  showmeans = True)
plt.title('Distribution of the number of unique chords found in each section', fontsize = 20)
plt.xticks([1,2,3], ['both', 'Chorus', 'Verse'], fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlabel('Section', fontsize = 18)
plt.ylabel('Number of different chord', fontsize = 18)
plt.show()


We observe that the number of chords appearing only in the chorus and in the verse and extremely similar in terms of distribution.

The ratio of the groups could inform us on the presence of subsets of one another. If more elements are found in the chorus than the verse then the length of the intersection is divided by the length of the chorus and vice versa if the verse is longer. This value is stored in the ratio column.  

Ratio = length(intersection)/length(chorus)

in the case that length(chorus)>length(verse)

One can observe no significant difference between the verse and chorus in termes of belonging to one another. And they share on average 80% of their chords.

In [ ]:
#Three sub datasets based on each category of majority
sub_chorus=intersection[intersection["majority"]=="chorus"]
sub_verse=intersection[intersection["majority"]=="verse"]
sub_equal=intersection[intersection["majority"]=="equal"]

In [ ]:
data_chords=[]
intersection["ratio"]=intersection.ratio.map(lambda y: y*100)
data_chords.append(sub_chorus.ratio)
data_chords.append(sub_verse.ratio)
data_chords.append(sub_equal.ratio)



plt.figure(figsize = (14,7))
plt.boxplot(data_chords, notch = True, 
                  bootstrap = 5000, 
                  showfliers = True, 
                  showmeans = True)
plt.title(' Comparison of number of unique chords shared by the two groups in each of the majority ', fontsize = 20)
plt.xticks([1,2,3], [ 'Chorus', 'Verse','No majority'], fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlabel('Majority', fontsize = 18)

plt.ylabel('Similarity of chorus and verse in %', fontsize = 18)
plt.show()

In [ ]:
plot_group(intersection,["verse_only","chorus_only","intersection"], title="Proportion of unique chords \
found in each section" )

In [ ]:
plot_group(intersection,["verse_only","chorus_only"],title="Proportion of unique chords \
found in each section - Intersection hidden" )
plot_group(intersection,["verse_only","chorus_only"],select_shorthand="maj", title="Proportion of unique chords \
found in each section - Major chords" )
plot_group(intersection,["verse_only","chorus_only"],select_shorthand="min",  title="Proportion of unique chords \
found in each section - Minor chords" )

In [ ]:
a=plot_group(sub_chorus,["chorus_only"],plot=False)
b=plot_group(sub_verse,["verse_only"],plot=False)
a=pd.concat([b,a],axis=1,sort=False)
a.columns=["verse_majority","chorus_majority"]
a.iplot(kind="bar",barmode="group",title="Comparison of chord distribution \
in Chorus and verse majority",xTitle="Chord name", yTitle="% of songs") 

a=plot_group(sub_chorus,["chorus_only"],select_shorthand="maj",plot=False)
b=plot_group(sub_verse,["verse_only"], select_shorthand="maj",plot=False)
a=pd.concat([b,a],axis=1,sort=False)
a.columns=["verse_majority","chorus_majority"]
a.iplot(kind="bar",barmode="group",title="Comparison of chord distribution \
in Chorus and verse majority - Major chords",xTitle="Chord name", yTitle="% of songs") 


a=plot_group(sub_chorus,["chorus_only"],select_shorthand="min",plot=False)
b=plot_group(sub_verse,["verse_only"], select_shorthand="min",plot=False)
a=pd.concat([b,a],axis=1,sort=False)
a.columns=["verse_majority","chorus_majority"]
a.iplot(kind="bar",barmode="group",title="Comparison of chord distribution \
in Chorus and verse majority - Minor chords",xTitle="Chord name", yTitle="% of songs") 


In the last two graphs we can observe that the chords found in the chorus majority subset have a tendency to be more major and less minor than in the verse majority. 

##### Comments: 
The results that we have found are more due to the difference between two types of songs (chorus-oriented and verse-oriented) than it is a result of true difference between the chorus and verse in them-selves, they are highly linked together inside songs and outside as well. We cannot ignore the fact that by reducing the initial group in three sub-parts we raise the probability of finding false results by raising the variance. As can be seen in the following graph, plotting only the songs where the diversity of chords is the same in both the chorus and verse we find differences conflicting with the previous ones. And thus the intrinsic singularity of chorus and verse cannot be outlined. We would need more data to further investigate the results and be able to find meaningfull conclusions. As for now we cannot from our dataset of chords draw any conclusion on how a chorus is made in regard to its verse or how different they are in general. 

In [ ]:
plot_group(sub_equal,["verse_only","chorus_only"],select_shorthand="maj", title="Proportion of unique chords \
found in verse and chorus with same diversity- Major chords " )
plot_group(sub_equal,["verse_only","chorus_only"],select_shorthand="min" ,title="Proportion of unique chords \
found in verse and chorus with same diversity- Minor chords")